<a href="https://colab.research.google.com/github/dajoeng/KDT_team4_FinalPJT/blob/main/01_%EC%9B%B9%ED%81%AC%EB%A1%A4%EB%A7%81/hangul_utils_%EA%B3%B0%EA%B3%B0_%EC%98%A4%ED%83%80%EA%B5%90%EC%A0%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive


In [2]:
!pip install symspellpy jamo hangul_utils
!pip install symspellpy-ko
!wget https://raw.githubusercontent.com/hermitdave/FrequencyWords/master/content/2018/ko/ko_50k.txt
from symspellpy import SymSpell, Verbosity
from symspellpy_ko import KoSymSpell, Verbosity
from hangul_utils import split_syllable_char, split_syllables, join_jamos
from jamo import h2j, j2hcj
import pandas as pd
import unicodedata

In [3]:
import pickle

In [ ]:
def to_jamos(text):
  return j2hcj(h2j(text))

# 만개의 레시피 재료 목록 자소 분리

In [6]:
with open('만개의레시피_재료_목록_오타처리용.pickle','rb') as f:
  vocab = pickle.load(f)

vocab = pd.DataFrame(vocab, index=['count']).T
vocab.reset_index(inplace=True)
vocab.columns = ["term", "count"]
vocab.head()

,term,count
0,현미,78
1,찹쌀,682
2,호두,450
3,물,24128
4,소금,32422


In [7]:
vocab.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38482 entries, 0 to 38481
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   term    38482 non-null  object
 1   count   38482 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 601.4+ KB


In [ ]:
vocab.term = vocab.term.map(to_jamos)
vocab.to_csv("만개의레시피_재료_목록_오타처리용_decomposed.txt", sep=" ", header=None, index=None)
vocab.head()

In [ ]:
# 저장했던 리스트 불러옴
with open('gom_typo.pkl','rb') as f:
  gom = pickle.load(f)

In [ ]:
for g in gom:
  idx = g.index('있어요')
  del g[0:idx + 1]
  
  if '일은' in g :
    idx = g.index('일은')
    del g[idx :]

  elif 'ㅇㅇ' in g :
    idx = g.index('ㅇㅇ')
    del g[idx :]
  
  else:
    pass

sym_spell = SymSpell(max_dictionary_edit_distance = 3)
dictionary_path = '만개의레시피_재료_목록_오타처리용_decomposed.txt'
sym_spell.load_dictionary(dictionary_path, 0, 1)

True

In [ ]:
# gom_typo_fix에 오타 교정된 재료 목록 저장
gom_typo_fix = []
for g in gom:
  g_typo_fix = []
  for term in g:
    term = split_syllables(term) 
    suggestions = sym_spell.lookup(term, Verbosity.ALL, max_edit_distance = 3)
    for sugg in suggestions :
      if sugg.distance < 2:
        typofix = join_jamos(sugg.term)
        g_typo_fix.append(typofix)
        break
      else:
        pass
  gom_typo_fix.append(g_typo_fix)


# gom_typo_fix를 다시 피클로 저장

In [ ]:
#gom_typo_fix를 다시 피클로 저장
with open('gom_typo_fix.pkl','wb') as f:
    pickle.dump(gom_typo_fix,f)
